In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 -q
!pip install yolov5 -q
!pip install datasets -q
!pip install --upgrade diffusers transformers accelerate datasets -q
!pip install accelerate transformers diffusers datasets safetensors -q
!pip install --upgrade diffusers -q

In [9]:
!pip show diffusers

Name: diffusers
Version: 0.31.0
Summary: State-of-the-art diffusion in PyTorch and JAX.
Home-page: https://github.com/huggingface/diffusers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/diffusers/graphs/contributors)
Author-email: diffusers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, importlib-metadata, numpy, Pillow, regex, requests, safetensors
Required-by: 


In [10]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Hoặc yolov5m, yolov5l, yolov5x

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-21 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [11]:
# Đường dẫn đến tập dữ liệu
dataset_path = "/content/drive/MyDrive/txttoimage11/dataset"

    """
    Phân tích ảnh trong tập dữ liệu và tạo nhãn.

    Args:
      dataset_path: Đường dẫn đến thư mục chứa tập dữ liệu.
      model: Mô hình YOLOv5.
    """

In [12]:
import os

def analyze_images(dataset_path, model):
    for filename in os.listdir(dataset_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(dataset_path, filename)
            label_path = os.path.splitext(image_path)[0] + ".txt"

            # Kiểm tra xem file nhãn đã tồn tại chưa
            if not os.path.exists(label_path):
                results = model(image_path)

                # Trích xuất đặc điểm đối tượng
                labels = []
                for *xyxy, conf, cls in results.xyxy[0]:
                    label = model.names[int(cls)]
                    labels.append(label)

                # Lưu nhãn vào file txt cùng tên với ảnh
                with open(label_path, "w") as f:
                    f.write(", ".join(labels))
            else:
                print(f"File nhãn đã tồn tại: {label_path}")

# Gọi hàm phân tích ảnh
analyze_images(dataset_path, model)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/txttoimage11/dataset'

In [ ]:
from torchvision import transforms

def preprocess(image):
    """
    Tiền xử lý ảnh: chuyển đổi ảnh PIL sang tensor và chuẩn hóa.
    """
    preprocess = transforms.Compose([
        transforms.Resize((512, 512)),  # Thay đổi kích thước ảnh
        transforms.ToTensor(),         # Chuyển ảnh PIL sang tensor
        transforms.Normalize([0.5], [0.5])  # Chuẩn hóa dữ liệu (-1 đến 1)
    ])
    return preprocess(image)


# chỉ CPU

In [ ]:
# import os
# import torch
# from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler # This line was causing the error
# from datasets import load_dataset
# from transformers import TrainingArguments, Trainer

# # Tải tập dữ liệu
# dataset = load_dataset("imagefolder", data_dir=dataset_path)

# # Khởi tạo pipeline Stable Diffusion
# pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2")
# pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe = pipe.to("cpu")

# # Đọc tất cả các nhãn trước
# labels_dict = {}
# for filename in os.listdir(dataset_path):
#     if filename.endswith(('.jpg', '.jpeg', '.png')):
#         label_path = os.path.join(dataset_path, os.path.splitext(filename)[0] + ".txt")
#         if os.path.exists(label_path):
#             with open(label_path, "r") as f:
#                 labels_dict[filename] = f.read()

In [ ]:
from datasets import load_dataset

# với gpu

In [ ]:
import os
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from transformers import TrainingArguments, Trainer

# Tải tập dữ liệu
dataset = load_dataset("imagefolder", data_dir=dataset_path)

# Khởi tạo pipeline Stable Diffusion
pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

# Đọc tất cả các nhãn trước
labels_dict = {}
for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        label_path = os.path.join(dataset_path, os.path.splitext(filename)[0] + ".txt")
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                labels_dict[filename] = f.read()

# Tiền xử lý ảnh

In [ ]:
# def preprocess_function(examples):
#     # Tiền xử lý ảnh
#     images = [preprocess(image.convert("RGB")) for image in examples["image"]]
#     examples["pixel_values"] = pipe.feature_extractor(images=images, return_tensors="pt").pixel_values

#     # Sử dụng nhãn từ YOLO
#     labels = []
#     for filename in examples["image_filepaths"]:
#         basename = os.path.basename(filename)  # Lấy tên file từ đường dẫn
#         label = labels_dict.get(basename, "")  # Truy cập nhãn từ labels_dict
#         labels.append(label)

#     examples["labels"] = labels
#     return examples


In [ ]:
def preprocess_function(examples):
    images = [preprocess(image.convert("RGB")) for image in examples["image"]]
    return {"pixel_values": torch.stack(images)}

In [ ]:
# Tiền xử lý dữ liệu
dataset = dataset.with_transform(preprocess_function)

In [ ]:
from torch.utils.data import DataLoader

class CustomDataset:
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image']  # Đọc ảnh dạng PIL
        if self.transform:
            image = self.transform(image)  # Áp dụng tiền xử lý
        label = item.get('text', '')  # Lấy prompt hoặc nhãn
        return image, label

# Tải tập dữ liệu và áp dụng tiền xử lý
custom_dataset = CustomDataset(dataset, transform=preprocess)


In [ ]:
def preprocess_function(examples):
    # ... (xử lý ảnh) ...

    # Sử dụng trực tiếp nhãn từ YOLO
    labels = []
    for image_path in examples["image_file_path"]:
        filename = os.path.basename(image_path)
        label = labels_dict.get(filename, "")
        labels.append(label)

    examples["labels"] = labels  # Gán nhãn vào examples["labels"]
    return examples
dataset = dataset.map(preprocess_function, batched=True, num_proc=4)

In [ ]:
custom_dataset = CustomDataset(dataset, transform=preprocess)

In [ ]:
# dataset = load_dataset(
#     "imagefolder",
#     data_dir=dataset_path,
#     split="train",
#     cache_dir="./cache"
# )

# # Thêm đường dẫn file ảnh vào examples
# dataset = dataset.map(lambda x: {"image_filepaths": x["image"]}, batched=False)


In [ ]:
# # Cấu hình huấn luyện
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir="./dreambooth_model",
#     num_train_epochs=4,
#     per_device_train_batch_size=1,
#     save_steps=500,
#     save_total_limit=2,
#     remove_unused_columns=False,
#     gradient_accumulation_steps=4,
#     fp16=True,
#     report_to="none"

# )

# # Huấn luyện với Trainer
# trainer = Trainer(
#     model=pipe.unet,  # Chỉ huấn luyện phần unet của mô hình
#     args=training_args,
#     train_dataset=dataset["train"],
# )

# trainer.train()


In [ ]:
# # Tham số huấn luyện
# training_args = TrainingArguments(
#     output_dir="path/to/save/model",
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     learning_rate=5e-6,
#     num_train_epochs=10,
#     save_strategy="epoch",
#     push_to_hub=False,
# )

# Chia dataset thành 80% train và 20% validation

In [ ]:
# import torch
# from torch.optim import AdamW
# from torch.utils.data import DataLoader
# from datasets import load_dataset

# # Tải tập dữ liệu
# train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# # Tối ưu hóa mô hình
# optimizer = AdamW(pipe.unet.parameters(), lr=5e-6)
# for epoch in range(10):
#     for batch in train_loader:
#         images = batch["image"]  # Dữ liệu ảnh
#         prompts = batch["text"]  # Prompt hoặc nhãn
#         outputs = pipe.unet(images)  # Huấn luyện mô hình U-Net
#         loss = outputs.loss  # Giả sử mô hình có tính toán loss
#         loss.backward()
#         optimizer.step()


#Xử lý toàn bộ dataset mà không chia phần

In [ ]:
# # Huấn luyện mô hình
# trainer = Trainer(
#     model=pipe,
#     args=training_args,
#     train_dataset=dataset,



In [ ]:
from tqdm.auto import tqdm

# CPU

In [ ]:
# # Tạo DataLoader
# train_dataloader = DataLoader(dataset["train"], batch_size=4, shuffle=True)

# # Tối ưu hóa
# optimizer = torch.optim.AdamW(pipe.unet.parameters(), lr=5e-6)

# # Vòng lặp huấn luyện
# num_epochs = 10
# for epoch in range(num_epochs):
#     for step, batch in enumerate(tqdm(train_dataloader)):
#         # Di chuyển dữ liệu sang GPU
#         batch = {k: v.to("cpu") for k, v in batch.items()}

#         # Tạo noise
#         noise = torch.randn(batch["pixel_values"].shape).to("cpu")

#         # Tạo timesteps
#         timesteps = torch.randint(0, pipe.scheduler.config.num_train_timesteps, (batch["pixel_values"].shape[0],)).long().to("cpu")

#         # Thêm noise vào ảnh
#         noisy_images = pipe.scheduler.add_noise(batch["pixel_values"], noise, timesteps)

#         # Dự đoán noise
#         noise_pred = pipe.unet(noisy_images, timesteps, batch["labels"]).sample

#         # Tính loss
#         loss = torch.nn.functional.mse_loss(noise_pred, noise)

#         # Cập nhật trọng số
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()



#GPU

In [ ]:
# Tạo DataLoader
train_dataloader = DataLoader(dataset["train"], batch_size=4, shuffle=True)

# Tối ưu hóa
optimizer = torch.optim.AdamW(pipe.unet.parameters(), lr=5e-6)

# Vòng lặp huấn luyện
num_epochs = 10
for epoch in range(num_epochs):
    for step, batch in enumerate(tqdm(train_dataloader)):
        # Di chuyển dữ liệu sang GPU
        batch = {k: v.to("cuda") for k, v in batch.items()}

        # Tạo noise
        noise = torch.randn(batch["pixel_values"].shape).to("cuda")

        # Tạo timesteps
        timesteps = torch.randint(0, pipe.scheduler.config.num_train_timesteps, (batch["pixel_values"].shape[0],)).long().to("cuda")

        # Thêm noise vào ảnh
        noisy_images = pipe.scheduler.add_noise(batch["pixel_values"], noise, timesteps)

        # Dự đoán noise
        noise_pred = pipe.unet(noisy_images, timesteps, batch["labels"]).sample

        # Tính loss
        loss = torch.nn.functional.mse_loss(noise_pred, noise)

        # Cập nhật trọng số
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()



# Lưu mô hình

In [ ]:
# # Lưu mô hình
# pipe.save_pretrained("path/to/save/model")

In [ ]:
# Lưu mô hình
pipe.save_pretrained("/content/drive/MyDrive/stable_diffusion_trained")

chuyển đổi ảnh thành định dạng base64 trước khi trả về.

In [ ]:
import base64
from io import BytesIO

def image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

@app.route('/generate', methods=['POST'])
def generate():
    description = request.form['description']
    prompt = f"{description}, no background"
    image = pipe(prompt).images[0]
    image_base64 = image_to_base64(image)
    return jsonify({'image': image_base64})


In [ ]:
from flask import Flask, request, jsonify
from diffusers import StableDiffusionPipeline

app = Flask(__name__)

# Tải mô hình Stable Diffusion đã huấn luyện
pipe = StableDiffusionPipeline.from_pretrained("path/to/save/model")
pipe = pipe.to("cuda")

@app.route('/generate', methods=['POST'])
def generate():
    description = request.form['description']
    # Xử lý mô tả và tạo prompt (ví dụ: thêm "no background")
    prompt = f"{description}, no background"
    image = pipe(prompt).images[0]
    # Chuyển đổi image sang base64 hoặc lưu vào file
    # ...
    return jsonify({'image': image_base64})  # Trả về ảnh

if __name__ == '__main__':
    app.run()